In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [12]:
df_1 = pd.read_csv('user1_data.csv')
df_2 = pd.read_csv('user2_data.csv')

In [3]:
X1 = df_1.drop(['sm','ttime'],axis=1)
y1 = df_1["sm"]

In [4]:
X2 = df_2.drop(['sm','ttime'],axis=1)
y2 = df_2["sm"]

In [40]:
# random forest with no depth limit
# current best 

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X1,y1,test_size=0.3,random_state=101)
rf = RandomForestRegressor(random_state=101)
rf.fit(X_train,y_train)
# rf.fit(X1,y1)
predictions = rf.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))

r2 = r2_score(y_test, predictions)
print("R^2 Score:", r2)



Mean Absolute Error: 115.17850581390579
Mean Squared Error: 254.86003753802856
R^2 Score: 0.9922637229882875


In [6]:
# decision tree classifier

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X1 = df_1.drop(['sm','ttime'],axis=1)
y1 = df_1["sm"]
X_train, X_test, y_train, y_test = train_test_split(X1,y1,test_size=0.2,random_state=42)
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))

Mean Absolute Error: 143.41380201602482
Mean Squared Error: 369.46027220411855


In [9]:
# ridge 

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge

X1 = df_1.drop(['sm','ttime'],axis=1)
y1 = df_1["sm"]

X_train, X_test, y_train, y_test = train_test_split(X1,y1,test_size=0.2,random_state=42)
model = Ridge()
model.fit(X_train,y_train)
predictions = model.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))





Mean Absolute Error: 1755.5256822935173
Mean Squared Error: 1951.2148880824834


In [32]:

corr_mat = df_1.corr()
print (corr_mat)


          pm1       pm2       pm3        am        sm        st       lum
pm1  1.000000  0.573090  0.552320  0.069879  0.441345 -0.122531  0.047982
pm2  0.573090  1.000000  0.922340  0.121761  0.728241 -0.205373  0.069458
pm3  0.552320  0.922340  1.000000  0.127813  0.730132 -0.185220  0.080850
am   0.069879  0.121761  0.127813  1.000000  0.148441 -0.287205  0.344563
sm   0.441345  0.728241  0.730132  0.148441  1.000000 -0.173479  0.082105
st  -0.122531 -0.205373 -0.185220 -0.287205 -0.173479  1.000000 -0.215903
lum  0.047982  0.069458  0.080850  0.344563  0.082105 -0.215903  1.000000


/tmp/ipykernel_6848/2268593849.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_mat = df_1.corr()


In [47]:
# Neural Net 92 % for 500 epoch

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.losses import Huber 
from keras.losses import LogCosh 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score


# load the data
# df_1 = pd.read_csv('data.csv')

# split into features (X) and target (y)
X = df_1.drop(['sm', 'ttime'], axis=1).values
y = df_1['sm'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

# define the model architecture
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))


# model = Sequential()
# model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dropout(0.2))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation=None))

# compile the model
# model.compile(loss='mean_squared_error', optimizer='adam') # 533
# model.compile(loss='mean_absolute_error', optimizer='adam') # 432 error
model.compile(loss=Huber(delta=1.0), optimizer='adam')  # 388
# model.compile(loss=LogCosh(), optimizer='adam') # 470


# Compile the model
# model.compile(loss='mean_squared_error', optimizer=Adam(lr=1)) # 886
# model.compile(loss='mean_squared_error', optimizer='adam') # 825

# train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


# make predictions on new data
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print("R^2 Score:", r2)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))

Epoch 1/50
339/339 [==============================] - 1s 2ms/step - loss: 3322.4543 - val_loss: 2560.9080
Epoch 2/50
339/339 [==============================] - 1s 2ms/step - loss: 1981.1218 - val_loss: 1893.9402
Epoch 3/50
339/339 [==============================] - 1s 2ms/step - loss: 1867.0162 - val_loss: 1861.7546
Epoch 4/50
339/339 [==============================] - 1s 2ms/step - loss: 1838.5505 - val_loss: 1812.9290
Epoch 5/50
339/339 [==============================] - 1s 2ms/step - loss: 1799.2489 - val_loss: 1799.3270
Epoch 6/50
339/339 [==============================] - 1s 2ms/step - loss: 1770.2373 - val_loss: 1770.9517
Epoch 7/50
339/339 [==============================] - 1s 2ms/step - loss: 1730.0536 - val_loss: 1690.8304
Epoch 8/50
339/339 [==============================] - 1s 2ms/step - loss: 1665.0768 - val_loss: 1589.4241
Epoch 9/50
339/339 [==============================] - 1s 2ms/step - loss: 1571.0598 - val_loss: 1528.4526
Epoch 10/50
339/339 [=========================

In [45]:
# 98 % accuracy for high epoch

X = df_1.drop(['sm', 'ttime'], axis=1)
y = df_1['sm']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

inputs = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
outputs = Dense(1, activation='linear')(x)

model = Model(inputs=inputs, outputs=outputs)
# model.summary()

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=500, batch_size=32)


predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))
print("R^2 Score:", r2)



Epoch 1/500
484/484 [==============================] - 2s 2ms/step - loss: 16333019.0000 - mse: 16333019.0000 - val_loss: 5876793.0000 - val_mse: 5876793.0000
Epoch 2/500
484/484 [==============================] - 1s 2ms/step - loss: 4533620.0000 - mse: 4533620.0000 - val_loss: 3827276.0000 - val_mse: 3827276.0000
Epoch 3/500
484/484 [==============================] - 1s 2ms/step - loss: 3146202.7500 - mse: 3146202.7500 - val_loss: 2458383.0000 - val_mse: 2458383.0000
Epoch 4/500
484/484 [==============================] - 1s 2ms/step - loss: 1818523.8750 - mse: 1818523.8750 - val_loss: 1425529.7500 - val_mse: 1425529.7500
Epoch 5/500
484/484 [==============================] - 1s 2ms/step - loss: 1219734.2500 - mse: 1219734.2500 - val_loss: 1137975.7500 - val_mse: 1137975.7500
Epoch 6/500
484/484 [==============================] - 1s 2ms/step - loss: 1038131.5625 - mse: 1038131.5625 - val_loss: 1009258.8750 - val_mse: 1009258.8750
Epoch 7/500
484/484 [==============================] - 1